# Setup

In [1]:
from openai import OpenAI
from openai import embeddings

from dotenv import load_dotenv
import os
import json

load_dotenv()

print(os.getenv("OPENAI_API_KEY")[:20], "...")
print(os.getenv("OPENAI_ORG")[:8], "...")
print(os.getenv("OPENAI_PROJECT")[:8], "...")

sk-proj-eUanLfMUnbXn ...
org-8d28 ...
proj_30j ...


## Envinronment Variables

Create a .env File: In the root of your project directory, create a file named `.env`. This file will hold your environment variables in the form of key-value pairs. For example:

```ini
    OPENAI_API_KEY = <openai_apikey>


    # optional 
    OPENAI_ORG = <openai_org_id>
    OPENAI_PROJECT = <openai_project_id>

```

In [2]:

# Set the API key and endpoint
api_key = os.getenv('OPENAI_API_KEY')
org = os.getenv('OPENAI_ORG')
project = os.getenv('OPENAI_PROJECT')
# model = "gpt-5-mini-2025-08-07"
model = "gpt-4.1-mini"

## OpenAI Client


In [3]:
client = OpenAI(
    api_key=api_key,
    organization=org,
    project=project
)

In [4]:
[m.id for m in client.models.list().data if m.id.startswith("gpt-4.1")]

['gpt-4.1-2025-04-14',
 'gpt-4.1',
 'gpt-4.1-mini-2025-04-14',
 'gpt-4.1-mini',
 'gpt-4.1-nano-2025-04-14',
 'gpt-4.1-nano']

## Responses API 

In [5]:
response = client.responses.create(
    model=model,
    input="Hello, world!"
)

print(response.output_text)

Hello! How can I assist you today?


In [6]:
input=[
    {
        "role": "user",
        "content": [

                { "type": "input_text", "text": "describe " },
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                }
        ]
    }
]
response = client.responses.create(
    model=model,
    input=input
)


In [7]:
print(response.output_text)

The image depicts a serene natural landscape featuring a wooden boardwalk that stretches into the distance, leading through a vast area of lush green grass. On either side of the boardwalk, tall grasses and some scattered bushes extend to the horizon. The sky above is expansive and mostly clear, with streaks of wispy clouds and a few scattered, fluffy clouds. The lighting suggests it could be either early morning or late afternoon, casting a soft, warm glow over the scene. The overall atmosphere is peaceful and inviting, ideal for a leisurely walk in nature.


## Responses API Streaming


In [8]:
response_stream = client.responses.create(
    model=model,
    input="write a long essage on a topic of your own!",
    stream=True
)


In [9]:
for event in response_stream:
    if event.type == 'response.output_text.delta':
        print(event.delta, end='')


Certainly! Here’s a thoughtful message on the importance of kindness in today’s world:

---

In a world that often feels fast-paced, competitive, and at times, divided, kindness remains one of the most powerful forces we possess. It’s easy to underestimate the impact a small act of kindness can have—whether it’s a genuine smile, a few encouraging words, or helping someone in need. Yet these simple gestures have the power to transform lives, build bridges between people, and create ripples that extend far beyond the immediate moment.

Kindness is universal; it transcends languages, cultures, and backgrounds. Everyone, regardless of who they are or where they come from, understands the warmth that comes from being treated with compassion and respect. In an era dominated by digital connections, where misunderstandings and negativity can spread rapidly, kindness acts as a balm—reminding us of our shared humanity.

Moreover, kindness benefits not only the receiver but also the giver. Resear

In [12]:
print(event.to_json())

{
  "response": {
    "id": "resp_68d5e4204ec481918fb54b0ad4346bac0cf1b91d7926f884",
    "created_at": 1758848032.0,
    "error": null,
    "incomplete_details": null,
    "instructions": null,
    "metadata": {},
    "model": "gpt-4.1-mini-2025-04-14",
    "object": "response",
    "output": [
      {
        "id": "msg_68d5e4209d9c81919f8529a2f736ed8e0cf1b91d7926f884",
        "content": [
          {
            "annotations": [],
            "text": "Certainly! Here’s a thoughtful message on the importance of kindness in today’s world:\n\n---\n\nIn a world that often feels fast-paced, competitive, and at times, divided, kindness remains one of the most powerful forces we possess. It’s easy to underestimate the impact a small act of kindness can have—whether it’s a genuine smile, a few encouraging words, or helping someone in need. Yet these simple gestures have the power to transform lives, build bridges between people, and create ripples that extend far beyond the immediate moment